In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
text=[]
with open('data/anna.txt', 'r') as f:
    text=f.read()
text[:102]

In [ ]:
chars=tuple(set(text))
char2int={ch:ii for ii, ch in enumerate(chars)}
int2char={ii:ch for ii, ch in enumerate(chars)}
encoded=np.array([char2int[ch] for ch in text])
print(len(text), len(encoded))

In [ ]:
def one_hot_encode(arr, n_labels):
    one_hot=np.zeros(shape=(arr.size, n_labels), dtype=np.float32)
    one_hot[np.arange(arr.size), arr.flatten()] = 1.
    return one_hot.reshape(*arr.shape, n_labels)

In [ ]:
def get_batches(arr, batch_size, seq_length):
    batch_area=batch_size*seq_length
    n_areas=(len(arr)-1)//batch_area
    arrx=arr[:n_areas*batch_area].reshape(batch_size, -1)
    arry=arr[1:n_areas*batch_area+1].reshape(batch_size, -1)
    for na in range(n_areas):
        start=na*seq_length
        end=start+seq_length
        x = arrx[:,start:end]
        y = arry[:,start:end]
        yield x, y

In [ ]:
class CharRNN(nn.Module):
    def __init__(self, n_input, n_hidden, n_layers, n_output):
        super(CharRNN, self).__init__()
        
        self.n_layers=n_layers
        self.n_hidden=n_hidden
        self.lstm=nn.LSTM(input_size=n_input, hidden_size=n_hidden, num_layers=n_layers, batch_first=True)
        self.fc=nn.Linear(n_hidden, n_output)
        
    def forward(self, x, hc):
        if not h:
            h0=torch.zeros((self.n_layers, x.shape[0], self.n_hidden), dtype=torch.float32)
            c0=torch.zeros((self.n_layers, x.shape[0], self.n_hidden), dtype=torch.float32)
            hc = (h0, c0)

        out,hcn=self.lstm(x, hc)
        out=out.contiguous().view(-1, self.n_hidden)
        out=self.fc(out)
        return out, hcn

In [ ]:
chars=tuple(set(text))
char2int={ch:ii for ii, ch in enumerate(chars)}
int2char={ii:ch for ii, ch in enumerate(chars)}
encoded=np.array([char2int[ch] for ch in text])

n_input=len(chars)
n_hidden=512
n_layers=2
n_output=len(chars)

net=CharRNN(n_input, n_hidden, n_layers, n_output)
print(net)

batch_size=128
seq_length=100
n_epochs=20
lr=0.001
val_frac=0.1
clip=5

net.train()
opt=torch.optim.Adam(net.parameters(), lr)
criterion=nn.CrossEntropyLoss()

val_idx=int(len(encoded)*val_frac)
data, val_data = encoded[:-val_idx], encoded[-val_idx:]

h=None

cnt=0
for x, y in get_batches(data, batch_size, seq_length):
    cnt+=1
    x=one_hot_encode(x, n_input)
    inputs, targets=torch.from_numpy(x), torch.from_numpy(y)
    
    opt.zero_grad()
    output, h = net(inputs, h)
    loss=criterion(output, targets.contiguous().view(-1).long())
    loss.backward()
    nn.utils.clip_grad_norm_(net.parameters(), clip)
    opt.step()
    
    